<a href="https://colab.research.google.com/github/barry80417/titanic-analysis/blob/main/titanic_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.gridspec as gridspec
df_train = pd.read_csv("https://raw.githubusercontent.com/agconti/kaggle-titanic/master/data/train.csv")
df_test = pd.read_csv("https://raw.githubusercontent.com/agconti/kaggle-titanic/master/data/test.csv")

用github內的資料直接讀進來

In [ ]:
#@title data information
df_train.head()
df_train.info()

 Link: https://www.kaggle.com/competitions/titanic/data

Survived: Survived (1) or died (0)

Pclass: Passenger’s class

Name: Passenger’s name

Sex: Passenger’s sex

Age: Passenger’s age

SibSp: define brother,sister,husband,wife...

Parch: define mother,father,daughter,son...

Ticket: ticket number

Fare:ticket fare

Cabin:cabin number

Embarked:port of embarkation (C=Cherbourg,Q=Queenstown,S=Southampton)

In [ ]:
#@title train data relation
#看訓練資料中是否有所關聯
corr = df_train.corr()
cmap = sns.diverging_palette(200, 0, as_cmap = True)
sns.heatmap(
    corr,
    cmap=cmap,
    square = False,
    cbar_kws = {'shrink': .9}, #顏色條大小
    ax = plt.subplots(figsize =(12,10))[1],
    annot = True,
    annot_kws = {'fontsize':12} #字體大小
)                             

In [ ]:
#@title survive or not pie plot
#kind(圖形類型)autopct小數
plt.figure(figsize = (10,5))
df_train['Survived'].value_counts().plot(kind = 'pie', colors =['lightcoral','skyblue'], autopct = '%1.2f%%')
plt.title('Survival')
plt.ylabel('')
plt.show()

In [ ]:
#@title survived data relation
#所有生還的人其他的資料是否有相關為看不出關聯
sur = (df_train['Survived']==1)
df_sur = df_train[sur].drop(['Survived'],axis = 1)
corr = df_sur.corr()
cmap = sns.diverging_palette(200, 0, as_cmap = True)
sns.heatmap(
    corr,
    cmap=cmap,
    square = False,
    cbar_kws = {'shrink': .9}, #顏色條大小
    ax = plt.subplots(figsize =(12,10))[1],
    annot = True,
    annot_kws = {'fontsize':12} #字體大小
)                             

In [ ]:
#@title survived data plot
selected_cols = ['Sex','Pclass','Embarked','SibSp','Parch']

plt.figure( figsize =(10,len(selected_cols)*5))
gs = gridspec.GridSpec(len(selected_cols),1)
for i, col in enumerate(df_train[selected_cols]):
  ax = plt.subplot(gs[i])
  sns.countplot(df_train[col], hue = df_train.Survived, palette = ['lightcoral','skyblue'])
  ax.set_yticklabels([])
  ax.set_ylabel('Counts')
  ax.legend(loc=1)#圖例顯示位子
  for p in ax.patches:
    ax.annotate('{:,}'.format(p.get_height()), (p.get_x(),p.get_height()+1.5))
plt.show()

In [24]:
df_train = df_train.drop(['Name','Ticket','Cabin'],axis = 1)

0                                Braund, Mr. Owen Harris
1      Cumings, Mrs. John Bradley (Florence Briggs Th...
2                                 Heikkinen, Miss. Laina
3           Futrelle, Mrs. Jacques Heath (Lily May Peel)
4                               Allen, Mr. William Henry
                             ...                        
886                                Montvila, Rev. Juozas
887                         Graham, Miss. Margaret Edith
888             Johnston, Miss. Catherine Helen "Carrie"
889                                Behr, Mr. Karl Howell
890                                  Dooley, Mr. Patrick
Name: Name, Length: 891, dtype: object

刪除看起來比較沒用的資料'Name','Ticket','Cabin'

In [80]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
df_train['Sex'] = labelencoder.fit_transform(df_train['Sex'])

看是否有空值

In [ ]:
df_train['Age'] = df_train['Age'].fillna(df_train['Age'].mode()[0])
df_train['Embarked'] = df_train['Embarked'].fillna(df_train['Embarked'].mode())
df_train['Embarked'] = labelencoder.fit_transform(df_train['Embarked'])
df_train = pd.get_dummies(df_train, columns = ['Embarked'])
df_train

將Age用眾數填補空值，Embarked用眾數填補空值

In [86]:
columns_X = set(df_train.columns) - {'Survived'}
columns_y = ['Survived']

train_X = df_train[columns_X]
train_y = df_train[columns_y]

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

log = LogisticRegression(random_state=0, max_iter=3000)
scores = cross_val_score(log, train_X, train_y.values.ravel(),cv=5,scoring='accuracy')
scores

array([0.7877095 , 0.78651685, 0.78089888, 0.76966292, 0.82022472])

In [87]:
df_train

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_0,Embarked_1,Embarked_2,Embarked_3
0,1,0,3,1,22.0,1,0,7.2500,0,0,1,0
1,2,1,1,0,38.0,1,0,71.2833,1,0,0,0
2,3,1,3,0,26.0,0,0,7.9250,0,0,1,0
3,4,1,1,0,35.0,1,0,53.1000,0,0,1,0
4,5,0,3,1,35.0,0,0,8.0500,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,1,27.0,0,0,13.0000,0,0,1,0
887,888,1,1,0,19.0,0,0,30.0000,0,0,1,0
888,889,0,3,0,24.0,1,2,23.4500,0,0,1,0
889,890,1,1,1,26.0,0,0,30.0000,1,0,0,0
